In [ ]:
# Function that infers the latency of a response from the lick rasters
def determine_latency(rxn, c):
    import numpy as np
    rx = []
    for a in range(c.shape[0]):
        r = (np.argwhere(c[a,:,:]== 1))

        if r.size == 0 and rxn[a] > 0:
            rx.append(rxn[a]);
        elif r.size == 0:
            rx.append(np.nan)
            #ch.append(np.nan)
        else:
            #ch.append(r[0][0])
            rx.append(r[0][1])

    return rx

In [ ]:
# Function to take a folder path, load the behavioral and licking data. Clean it to a nice lil pandas df
def load_beh_data(path, animal_id):

    import os 
    import pandas as pd
    import numpy as np

    file_pandas = animal_id + "_df.csv"
    if os.path.isfile(file_pandas):
        beh_df = pd.read_csv(file_pandas, index = False)
    else:
    
        file_data = 'analyze_animal_'+animal_id+'.mat'
        data_path = os.path.join(path,file_data)

        beh_data = import_matlab_file(data_path)
        beh_df, ignore = pd_from_uneven_dict(beh_data['animal'])
        beh_df = beh_df.rename(columns={"lick_0": "RH", "lick_1": "LH", "lick_2": "RM", "lick_3": "LM", "lick_4": "NG"})

        boo_cols = ["RH", "LH", "RM", "LM", "NG", "LED"]
        beh_df[boo_cols] = beh_df[boo_cols].astype(bool)


        file_ttl = 'ttl_info_'+animal_id+'.mat'
        ttl_path = os.path.join(path, file_ttl)
        ttl_data = import_matlab_file(ttl_path)

        beh_df['Spout_1']= np.mean(ttl_data['trial_info']['lick'][:,750:825,0],axis = 1)
        beh_df['Spout_2'] = np.mean(ttl_data['trial_info']['lick'][:,750:825,1],axis = 1)


        file_ttl2 = 'ttl_choices_'+animal_id+'.mat'
        ttl_path2 = os.path.join(path, file_ttl2)
        ttl_data2 = import_matlab_file(ttl_path2)
        c = ttl_data2['trial_info']['choice'][:,:,750:1750]
        beh_df['latency'] = determine_latency(beh_df['rxnTime'], c)

        beh_df.to_csv(file_pandas, index= False)
        

    return beh_df

In [ ]:
# function to take beh_df and processes the data into the form for modeling. Keep as a dataframe to 
# allow for easier subsampling of trials. Next steps: incorporate a function that allows for easy sampling
# of trial history
def clean_beh_data(b):
    import numpy as np
    import pandas as pd

    
    b2 = pd.DataFrame()
    ns = np.array(np.append(1, np.diff(b['sessionNum'])), dtype = bool)

    b2['Rightward'] = (b['RH'] | b['LM']).astype(int)
    pr = np.concatenate(([0], b2['Rightward'][:-1].values));
    pr[ns] = 0;
    b2['Previous_rightward'] = pr;

    b2['Correct'] = (b['RH'] | b['LH']).astype(int)
    pc = np.concatenate(([0], b2['Correct'][:-1].values))  
    pc[ns] = 0;
    b2['Previous_correct'] = pc;

    b2['Stimulus'] =np.log2(b['stimulus']/8)/2
    b2['LED'] = b['LED'].astype(int);
    b2['Target'] = b['target'].astype(int);
    b2['Session'] = b['sessionNum'];

   
    b2['Latency'] = b['latency']/1000;
    #b2['New_session']= ns;
   
    go = (b['RH'] | b['LM'] | b['RM'] | b['LH']).values
    
    b2 = b2.iloc[go,:]
    
    return b2

In [ ]:
# function to arrange behavioral data in the format required for psytrack
def preprocess_psytrack(b2, animal_id):
    import numpy as np

    inputs = dict(stim = np.expand_dims(np.array(b2['Stimulus']), axis=1),
        LED = np.expand_dims(np.array(b2['Stimulus']).astype(int), axis=1),
        previous_correct =  np.expand_dims(np.array(b2['Previous_correct']).astype(int), axis=1),
        previous_choice =  np.expand_dims(np.array(b2['Previous_rightward']).astype(int), axis=1),
        latency = np.expand_dims (np.array(b2['Previous_rightward']).astype(int), axis=1)
    )   


    dat = dict(
            subject = animal_id,
            inputs = inputs,
            correct = np.array(b2['Correct']),
            answer = np.array(b2['Target']),
            y = np.array(b2['Rightward']),
            dayLength=np.array(b2.groupby(['Session']).size()),
        )


    return dat